---
# 0. 라이브러리 
---

In [1]:
# 문장을 토큰화하는 모듈 설치
!pip3 install torchtext==0.10.0
!pip3 install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 831.4/831.4 MB 2.0 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1+cu116
    Uninstalling torch-1.13.1+cu116:
      Successfully uninstalled torch-1.13.1+cu116
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.14.1
    Uninstalling torchtext-0.14.1:
      Successfully uninstalled torchtext-0.14.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.14.1+cu116 requires torch==1.13.1, but you have torch 1.9.0 which is incompatible.
torchaudio 0.13.1+cu116 requires torch==1.13.1, but you have torch 1.9.0 which is incompatible.
Looking in indexes: https://pypi.org/

In [2]:
import os 
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator
import spacy
import numpy as np

import random
import math
from tqdm.notebook import tqdm

from torchsummary import summary as summary_
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

---
# 1. Data
----

In [4]:
# 문장을 토큰화하는 모델 로드
spacy_de = spacy.load('de_core_news_sm')
spacy_en = spacy.load('en_core_web_sm')

In [5]:
# tokenizer function 생성
def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)][::-1]

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

#### Field 정의

- sequential : 시퀀스 데이터 여부. (True가 기본값)
- use_vocab : 단어 집합을 만들 것인지 여부. (True가 기본값)
- tokenize : 어떤 토큰화 함수를 사용할 것인지 지정. (string.split이 기본값)
- lower : 영어 데이터를 전부 소문자화한다. (False가 기본값)
- batch_first : 미니 배치 차원을 맨 앞으로 하여 데이터를 불러올 것인지 여부. (False가 기본값)
- is_target : 레이블 데이터 여부. (False가 기본값)
- fix_length : 최대 허용 길이. 이 길이에 맞춰서 패딩 작업(Padding)이 진행된다.

In [6]:
# torchtext의 Field는 데이터를 어떻게 처리할지 조절합니다.
SRC = Field(tokenize = tokenize_de, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

TRG = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

---
### 1-1. Data Load
----

In [7]:
# 30,000개의 영어, 독일, 프랑스어 문장을 포함합니다.
train_data, valid_data, test_data = Multi30k.splits(exts=('.de', '.en'), fields=(SRC,TRG))

downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:01<00:00, 700kB/s] 


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 202kB/s]


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 192kB/s]


In [8]:
print('Train 개수:', len(train_data))
print('Test 개수:', len(test_data))
print('Valid 개수:', len(valid_data))

Train 개수: 29000
Test 개수: 1000
Valid 개수: 1014


In [9]:
print(vars(train_data.examples[0]))

{'src': ['.', 'büsche', 'vieler', 'nähe', 'der', 'in', 'freien', 'im', 'sind', 'männer', 'weiße', 'junge', 'zwei'], 'trg': ['two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.']}


---
### 1-2. Vocab 생성
----

- min_freq = 해당 인수를 사용하여 n회 이상 나타내는 토큰만 어휘에 표시   
   n-1회 나타날 경우 <UNK\>으로 표시

In [10]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [11]:
print('Train Vocab의 수:', len(SRC.vocab))
print('Test Vocab의 수:', len(TRG.vocab))

Train Vocab의 수: 7853
Test Vocab의 수: 5893


In [12]:
batch_size = 128

# Iterator를 통해 원본 문장과 배치 문장과 동일하게 패딩 처리
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), batch_size=batch_size, device=device)

---
# 2. Model
----

<img src = 'https://raw.githubusercontent.com/bentrevett/pytorch-seq2seq/49df8404d938a6edbf729876405558cc2c2b3013//assets/seq2seq1.png'>


---
### 2-1. Encoder
----

<img src='https://raw.githubusercontent.com/bentrevett/pytorch-seq2seq/49df8404d938a6edbf729876405558cc2c2b3013//assets/seq2seq5.png'>

- 각 레이어 사이에서 Dropout이 적용되기 때문에 GRU를 사용시에는 미적용 

- GRU에는 LSTM과 같은 cell state가 없기 때문에 hidden state만 반환

In [13]:
class Encoder(nn.Module):
  def __init__(self, input_dim, emb_dim, hid_dim, dropout):
    super().__init__()

    self.hid_dim = hid_dim
    
    self.embedding = nn.Embedding(input_dim, emb_dim)
    self.gru = nn.GRU(emb_dim, hid_dim) # 각 레이어 사이에서 드롭아웃이 적용되기 때문에 여기서는 미적용

    self.dropout = nn.Dropout(dropout)

  def forward(self, src):
    out = self.embedding(src)  # src = [src_len , batch_size]
    embedded = self.dropout(out) # embedded = [src_len, batch_size, emb_dim]
                                 # src를 dense 벡터에 매핑하는 임베딩 레이어의 출력
    out, hidden = self.gru(embedded)

    # out = [src_len, barch_size, hid_dim] / LSTM의 출력
    # hidden = [n_layer, batch_size, hid_dim] / LSTM의 hidden state
    # cell = [n_layer, batch_size, hid_dim] / LSTM의 cell state
    
    return hidden
    

---
### 2-2. Decoder
----

<img src='https://raw.githubusercontent.com/bentrevett/pytorch-seq2seq/49df8404d938a6edbf729876405558cc2c2b3013//assets/seq2seq6.png'>

- Decoder의 GRU가 embedded된 토큰과 이전의 hidden state만을 취하는 대신 context 벡터도 취함 

- Decoder의 모든 time step에서 Encoder가 반환한 동일한 context 벡터를 다시 사용한다. 

- LSTM을 사용했을 경우 해당 time step에서 최상위 계층의 Decoder의 hidden state만을 사용하여 Linear 계층을 통해 다음을 예측했다면

- 현재는 현재 토큰과 context 벡터의 embedding도 Linear 계층에 전달한다.

In [33]:
class Decoder(nn.Module):
  def __init__(self, output_dim, emb_dim, hid_dim, dropout):
    super().__init__()

    self.output_dim = output_dim
    self.hid_dim = hid_dim 

    self.embedding = nn.Embedding(output_dim, emb_dim)

    self.gru = nn.GRU(emb_dim + hid_dim, hid_dim) 
    '''emb_dim + hid_dim을 해주는 이유는 Decoder의 입력의 현재 embedded 단어와 Encoder 출력의 context 벡터를 모두 고려하기 위해 사용 
       context 벡터는 Decoder가 대상 문장을 생성하기 위해 사용하는 소스 문장에 대한 정보를 나타냄 
       따라서. 현재 Embedded 단어와 context 벡터를 모두 연결함으로써 Decoder는 각 대상 단어를 생성할 때 현재 입력과 소스 문장 정보를 모두 고려할 수 있게 된다.'''

    self.fc_layer = nn.Linear(emb_dim + hid_dim * 2, output_dim)
    '''emb_dim + hid_dim * 2의 이유는 해당 Layer의 입력은 embedded된 입력과 GRU 계층의 hidden state를 연결하기 떄문
       Seq2Seq 모델에서 Decoder는 예측을 하기 위해 입력 seq 정보와 hidden state가 모두 필요하다.
       embedded된 입력은 GRU를 통해 전달되고 시퀀스 정보를 제공하며, hidden state는 context 벡터를 제공한다.
       embedded된 입력과 hidden state를 모두 연결함으로써 Decoder는 seq 및 context 정보에 접근할 수 있게되어 더 많은 정보에 입각하여 예측값을 추론할 수 있게된다.
       
       즉, hid_dim * 2는 GRU 계층의 hidden state가 hidden state와 cell state의 튜플이기 때문에 둘을 연결함으로써 Linear 계층에 공급하기 위해 두 상태의 정보를 결합하는 것이다.
       
       GRU계층의 hidden state가 tuple이라는 것은 hidden state가 단일값이 아닌 두 개의 별도의 값으로 표현된다는 뜻 
       2개의 값은 hidden state의 memory cell과 activation 벡터를 나타내는데 사용
       두개의 별도 값을 사용하면 GRU 계층이 더 오랜 시간 정보를 유지하고 순차적 데이터에서 장기 종속성을 처리하는 모델의 능력을 항샹시킬 수 있다.'''

    self.dropout = nn.Dropout(dropout)

  def forward(self, input, hidden, context)  :
    
    # input = [batch_size]
    # hidden = [n_layer, batch_size, hid_dim]
    # cell = [n_layer, batch_size, hid_dim] 
    input = input.unsqueeze(0) # [1, batch_size]

    out = self.embedding(input) # embedded = [1, batch_size, emb_dim]
    embedded = self.dropout(out)

    emb_con = torch.cat((embedded, context), dim = 2)

    output, hidden = self.gru(emb_con, hidden)
    # Decoder의 Sequence_length는 각 time step에 대한 입력이 단일 word/torken만 처리하기 때문에 항상 값은 1이다.
    # output = [seq len, batch size, hid dim]   ->  [1, batch size, hid dim]
    # hidden = [n layers, batch size, hid dim]  ->  [n layers, batch size, hid dim]

    output = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), dim = 1)

    predict = self.fc_layer(output) # predict = [batch_size, output_dim]

    return predict, hidden

---
### 2-3. Seq2Seq
---

<img src='https://raw.githubusercontent.com/bentrevett/pytorch-seq2seq/49df8404d938a6edbf729876405558cc2c2b3013//assets/seq2seq7.png'>

1. input 문장 받기

2. Encoder를 이용해 context 벡터 생성 

3. Decoder를 이용해 예측 output 문장 생성 

- assert 함수

  - 가정 설정문 

  - 디버깅 목적으로 사용, 일반적으로 입력 인수의 유효성을 검사하거나 프로그램의 내부 상태를 검증하는데 사용

  - 특정 조건이 충족되는지 확인하고, 조건이 True이면 아무것도 발생X, 미충족시 AssertionError가 발생

In [34]:
class Seq2Seq(nn.Module):
  def __init__(self, encoder, decoder, device):
    super().__init__()

    self.encoder = encoder
    self.decoder = decoder 
    self.device = device

    assert encoder.hid_dim == decoder.hid_dim, 'Hidden dimensions of encoder decoder must be equal'



  def forward(self, src, trg, teacher_forcing_ratio = 0.5):
                  # src = [src_len, batch_size]
                  # trg = [trg_len, batch_size]

    batch_size = trg.shape[1]
    trg_len = trg.shape[0] # 타겟 토큰 길이 
    trg_vocab_size = self.decoder.output_dim # context vector의 차원
    
    # Decoder의 출력을 저장할 Tensor
    outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)


    # Encoder의 Hidden state를 Decoder의 초기 hidden state로 사용
    context = self.encoder(src)
    hidden = context
    
    # Decoder의 첫 번째 input을 <token>으로 사용
    input = trg[0,:]


    '''한번에 batch_size만큼의 token들을 독립적으로 계산
    즉, 총 trg_len번의 for문이 돌아가며 이 for문이 다 돌아가야지만 하나의 문장이 decoding됨
    또한, 1번의 for문당 128개의 문장의 각 token들이 다같이 decoding되는 것'''
    
    for i in range(1, trg_len):
      
      output, hidden = self.decoder(input, hidden, context)

      # 예측값 저장 
      outputs[i] = output

      # teacher forcing 사용 여부
      teacher_force = random.random() < teacher_forcing_ratio

      # 가장 높은 확률 값 얻기
      top1 = output.argmax(1)

      # teacher_forcing의 경우 다음 LSTM에 target token 입력
      input = trg[i] if teacher_force else top1

    return outputs


---
# 3. Model setting
----

In [35]:
# 하이퍼 파라미터 지정
input_dim = len(SRC.vocab)
output_dim = len(TRG.vocab)
enc_emb_dim = 256 # 임베딩 차원
dec_emb_dim = 256
hid_dim = 512 # hidden state 차원
enc_dropout = 0.5
dec_dropout = 0.5

In [36]:
# 모델 생성
enc = Encoder(input_dim, enc_emb_dim, hid_dim, enc_dropout)
dec = Decoder(output_dim, dec_emb_dim, hid_dim, dec_dropout)

model = Seq2Seq(enc, dec, device).to(device)

In [37]:
model

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7853, 256)
    (gru): GRU(256, 512)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256)
    (gru): GRU(768, 512)
    (fc_layer): Linear(in_features=1280, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [38]:
# 가중치 초기화
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

model.apply(init_weights)

# 모델의 학습가능한 파라미터 수 측정
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainableparameters')

The model has 14,219,781 trainableparameters


In [39]:
# optimizer
optimizer = optim.Adam(model.parameters())

# loss function
# pad에 해당하는 index는 무시합니다.
trg_pad_idx = TRG.vocab.stoi[TRG.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index=trg_pad_idx)

---
# 4. Train
---

In [40]:
def train(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0

    for i, batch in enumerate(tqdm(iterator)):
        src = batch.src
        trg = batch.trg
        optimizer.zero_grad()

        output = model(src,trg) # [trg len, batch size, output dim]
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim) # loss 계산을 위해 1d로 변경
        trg = trg[1:].view(-1) # loss 계산을 위해 1d로 변경

        loss = criterion(output, trg)
        loss.backward()
        # 기울기 clip
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [41]:
def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    
    with torch.no_grad():
        for i, batch in enumerate(tqdm(iterator)):
            src = batch.src
            trg = batch.trg

            # output: [trg len, batch size, output dim]
            output = model(src, trg, 0) # teacher forcing off
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim) # [(trg len -1) * batch size, output dim]
            trg = trg[1:].view(-1) # [(trg len -1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [42]:
num_epochs = 10
clip = 1

best_valid_loss = float('inf')

for epoch in range(num_epochs):
    
    train_loss = train(model, train_iterator, optimizer, criterion, clip)
    valid_loss = evaluate(model, valid_iterator, criterion)
      
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

  0%|          | 0/227 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

	Train Loss: 4.802 | Train PPL: 121.715
	 Val. Loss: 4.637 |  Val. PPL: 103.261


  0%|          | 0/227 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

...

  0%|          | 0/8 [00:00<?, ?it/s]

	Train Loss: 2.009 | Train PPL:   7.453
	 Val. Loss: 3.575 |  Val. PPL:  35.699


  0%|          | 0/227 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

	Train Loss: 1.863 | Train PPL:   6.440
	 Val. Loss: 3.637 |  Val. PPL:  37.985


In [43]:
# best val loss일 때의 가중치를 불러옵니다.
model.load_state_dict(torch.load('tut1-model.pt'))

# test loss를 측정합니다.
test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

  0%|          | 0/8 [00:00<?, ?it/s]

| Test Loss: 3.490 | Test PPL:  32.797 |


Loss값만 보더라도 이전의 LSTM을 활용한 모델보다는 더 좋은 성능을 나타나는것을 알 수 있다.